In [11]:
import pandas as pd
import psycopg2
conn = psycopg2.connect(database = 'ad_model',
            user = 'ads_user',
            password = 'ads_pass',
            host="blaze-ad-model-rds-aur-cls-prod-ind-ap-southeast-1."
            "cluster-cka7qtmp9a6h.ap-southeast-1.rds.amazonaws.com",
            port = 5432)

cur = conn.cursor()

In [93]:
try:
    cur.execute('''
        select
            audience_targeting_rule.ad_set_id,
            audience_targeting_rule_clause.rule_type,
            attribute.name,
            attribute_value.name,
            attribute_value.enabled,
            attribute_value.ssai_tag,
            ad_set.name,
            ad_set.impression_target,
            ad_set_impr.impr,
            asset.duration / 1000,
            ad_set_impr.min_cd,
            ad_set_impr.max_cd,
            ad_set.deleted,
            ad_set.enabled,
            ad_set.priority
        from audience_targeting_rule
        join audience_targeting_rule_clause
        on audience_targeting_rule.id = audience_targeting_rule_clause.audience_targeting_rule_id
        join audience_targeting_rule_clause_value
        on audience_targeting_rule_clause.id = audience_targeting_rule_clause_value.audience_targeting_rule_clause_id
        join attribute
        on audience_targeting_rule_clause.attribute_id = attribute.id
        join attribute_value
        on audience_targeting_rule_clause_value.attribute_value_id = attribute_value.id
        join ad_set
        on ad_set.id = audience_targeting_rule.ad_set_id
        left join ad
        on ad.ad_set_id = audience_targeting_rule.ad_set_id
        left join asset
        on ad.id = asset.ad_id
        join (
            select ad_set_id, sum(num) as impr, min(cd) as min_cd, max(cd) as max_cd
            from campaign_impression_statistics
            group by ad_set_id
        ) ad_set_impr
        on audience_targeting_rule.ad_set_id = ad_set_impr.ad_set_id
    ''')
    df=pd.DataFrame(cur.fetchall(), columns=['ad_set_id', 'rule_type', 'attr_name', 'attr_val', 'attr_enabled', 'ssai', 'ad_set', 'impr_goal', 'impr', 'duration', 'min_cd', 'max_cd', 'ad_set_deleted', 'ad_set_enabled', 'priority'])
except Exception as e:
    print(e)
    conn.rollback()

df

,ad_set_id,rule_type,attr_name,attr_val,attr_enabled,ssai,ad_set,impr_goal,impr,duration,min_cd,max_cd,ad_set_deleted,ad_set_enabled,priority
0,5345,Include,city,Hyderabad,True,M_HYD,SSCR-20-000249_Fitternity_PunHyd_10,2083333,793229,10.0,2019-12-21,2020-01-18,False,True,3
1,5345,Include,city,Pune,False,M_PUN,SSCR-20-000249_Fitternity_PunHyd_10,2083333,793229,10.0,2019-12-21,2020-01-18,False,True,3
2,3548,Include,state,Punjab + Haryana,False,S_PBHR,Rapido_Punjab_10,2100000,15827,11.0,2019-10-02,2019-10-05,False,False,1
3,3548,Include,state,Punjab + Haryana,False,S_PBHR,Rapido_Punjab_10,2100000,15827,9.0,2019-10-02,2019-10-05,False,False,1
4,4019,Include,customAudience,"Device - Over Rs. 20,000",False,C2_1,Apple_Midroll_30,50000000,5340234,29.0,2019-11-03,2019-12-01,False,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74493,4588,Exclude,state,Karnataka,True,S_KA,_Winzo_Sept_spot_15,10000000,1,NaN,2020-01-17,2020-01-17,True,True,4
74494,4588,Exclude,state,Kerala,True,S_KL,_Winzo_Sept_spot_15,10000000,1,NaN,2020-01-17,2020-01-17,True,True,4
74495,4588,Exclude,state,MaharashtraX,False,S_MHX,_Winzo_Sept_spot_15,10000000,1,NaN,2020-01-17,2020-01-17,True,True,4
74496,4317,Include,city,Kolkata,True,M_KOL,Men's Bengali_6,1000000,3,NaN,2020-01-10,2020-01-10,True,True,5


In [94]:
df.nunique()

ad_set_id         5544
rule_type            2
attr_name           19
attr_val            82
attr_enabled         2
ssai                82
ad_set            5166
impr_goal         2740
impr              5535
duration            46
min_cd             420
max_cd             469
ad_set_deleted       2
ad_set_enabled       2
priority             5
dtype: int64

In [99]:
pd.to_datetime(df.min_cd).describe(datetime_is_numeric=True)

count                            74498
mean     2022-02-16 05:22:56.155064576
min                2019-07-01 00:00:00
25%                2021-10-10 00:00:00
50%                2022-03-26 00:00:00
75%                2022-09-04 00:00:00
max                2023-03-13 00:00:00
Name: min_cd, dtype: object

In [115]:
(pd.to_datetime(df.max_cd)-pd.to_datetime(df.min_cd)).describe(datetime_is_numeric=True)

count                         74498
mean     18 days 01:19:30.103895406
std      22 days 18:03:00.212629039
min                 0 days 00:00:00
25%                 5 days 00:00:00
50%                13 days 00:00:00
75%                24 days 00:00:00
max              1300 days 00:00:00
dtype: object

In [122]:
df[(pd.to_datetime(df.max_cd)-pd.to_datetime(df.min_cd)).map(lambda x: x.days) > 300]

,ad_set_id,rule_type,attr_name,attr_val,attr_enabled,ssai,ad_set,impr_goal,impr,duration,min_cd,max_cd,ad_set_deleted,ad_set_enabled,priority
724,5090,Include,customAudience,Device - Unkown,False,C2_NA,SSCR-20-000093_Apple_Snowbrawl_Jio_30,1000000,2252311,29.0,2019-12-12,2021-08-18,False,True,3
725,5090,Include,customAudience,"Device - Over Rs. 20,000",False,C2_1,SSCR-20-000093_Apple_Snowbrawl_Jio_30,1000000,2252311,29.0,2019-12-12,2021-08-18,False,True,3
797,5092,Include,customAudience,Device - Unkown,False,C2_NA,SSCR-20-000095_Apple_Snowbrawl_Vodafone_30,1000000,524487,29.0,2019-11-25,2021-08-18,False,True,3
798,5092,Include,customAudience,"Device - Over Rs. 20,000",False,C2_1,SSCR-20-000095_Apple_Snowbrawl_Vodafone_30,1000000,524487,29.0,2019-11-25,2021-08-18,False,True,3
1285,5340,Exclude,state,MaharashtraX,False,S_MHX,SSCR-20-000244_PlayRummy_MindGame_20,1,59941384,19.0,2019-08-21,2023-03-13,False,True,3
1286,5340,Exclude,state,Telangana,False,S_TG,SSCR-20-000244_PlayRummy_MindGame_20,1,59941384,19.0,2019-08-21,2023-03-13,False,True,3
1287,5340,Exclude,state,Kerala,True,S_KL,SSCR-20-000244_PlayRummy_MindGame_20,1,59941384,19.0,2019-08-21,2023-03-13,False,True,3
1288,5340,Exclude,state,Gujarat,False,S_GJ,SSCR-20-000244_PlayRummy_MindGame_20,1,59941384,19.0,2019-08-21,2023-03-13,False,True,3
1606,5472,Exclude,state,Odisha+North East,True,S_ORNE,PlayRummy_MindGames_State_20,30000000,71416020,19.0,2019-11-02,2021-08-28,False,True,3
1607,5472,Exclude,state,Telangana,False,S_TG,PlayRummy_MindGames_State_20,30000000,71416020,19.0,2019-11-02,2021-08-28,False,True,3


In [102]:
df.duration.describe()

count    69430.000000
mean        18.153161
std          7.691597
min          3.000000
25%         10.000000
50%         15.000000
75%         24.000000
max        120.000000
Name: duration, dtype: float64

In [95]:
df[df.ad_set.duplicated()&~df.ad_set_id.duplicated()].sort_values('ad_set')

,ad_set_id,rule_type,attr_name,attr_val,attr_enabled,ssai,ad_set,impr_goal,impr,duration,min_cd,max_cd,ad_set_deleted,ad_set_enabled,priority
34243,15125,Include,city,1MnPlusTown,True,M_1PT,1XBET_1XBET_Youraj_Mobile_1MnplusTowns_10,600000,361674,10.0,2022-08-27,2022-09-09,False,True,3
63439,14002,Include,platform,iOS,True,P_P_IOS,ABFRL_LoiusPhilippe_PermaPress_Eng_Mobile_ROS_15,11654704,12841299,15.0,2022-04-01,2022-04-03,False,True,3
63443,14003,Include,platform,iOS,True,P_P_IOS,ABFRL_LoiusPhilippe_PermaPress_Eng_Mobile_ROS_15,147310131,131449252,15.0,2022-04-03,2022-05-15,False,True,3
63447,14035,Include,platform,iOS,True,P_P_IOS,ABFRL_LoiusPhilippe_PermaPress_Eng_Mobile_ROS_15,57312628,58229240,15.0,2022-05-17,2022-05-30,False,True,3
62164,14025,Include,platform,iOS,True,P_P_IOS,ABFRL_VanHeusen_Run_Eng_Mobile_ROS_10,56576438,56588049,9.0,2022-04-09,2022-05-11,False,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30751,16385,Include,platform,iOS,True,P_P_IOS,actyv_ai_actyv_ai_Mobile_ROS_10,13032581,13151331,10.0,2023-01-03,2023-01-07,False,True,3
23558,15595,Include,platform,iOS,True,P_P_IOS,actyv_ai_actyv_ai_Mobile_ROS_10,9523809,9802862,9.0,2022-09-28,2022-10-05,False,True,3
14854,14889,Include,platform,iOS,True,P_P_IOS,actyv_ai_actyv_ai_Mobile_ROS_10,8400000,7071595,10.0,2022-06-12,2022-06-19,False,True,3
25272,15019,Include,platform,iOS,True,P_P_IOS,actyv_ai_actyv_ai_Mobile_ROS_10,18045112,18075536,9.0,2022-08-27,2022-09-11,False,True,3


In [103]:
df2 = df[~df.ad_set.str.lower().str.contains('test')]
df2

,ad_set_id,rule_type,attr_name,attr_val,attr_enabled,ssai,ad_set,impr_goal,impr,duration,min_cd,max_cd,ad_set_deleted,ad_set_enabled,priority
0,5345,Include,city,Hyderabad,True,M_HYD,SSCR-20-000249_Fitternity_PunHyd_10,2083333,793229,10.0,2019-12-21,2020-01-18,False,True,3
1,5345,Include,city,Pune,False,M_PUN,SSCR-20-000249_Fitternity_PunHyd_10,2083333,793229,10.0,2019-12-21,2020-01-18,False,True,3
2,3548,Include,state,Punjab + Haryana,False,S_PBHR,Rapido_Punjab_10,2100000,15827,11.0,2019-10-02,2019-10-05,False,False,1
3,3548,Include,state,Punjab + Haryana,False,S_PBHR,Rapido_Punjab_10,2100000,15827,9.0,2019-10-02,2019-10-05,False,False,1
4,4019,Include,customAudience,"Device - Over Rs. 20,000",False,C2_1,Apple_Midroll_30,50000000,5340234,29.0,2019-11-03,2019-12-01,False,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74493,4588,Exclude,state,Karnataka,True,S_KA,_Winzo_Sept_spot_15,10000000,1,NaN,2020-01-17,2020-01-17,True,True,4
74494,4588,Exclude,state,Kerala,True,S_KL,_Winzo_Sept_spot_15,10000000,1,NaN,2020-01-17,2020-01-17,True,True,4
74495,4588,Exclude,state,MaharashtraX,False,S_MHX,_Winzo_Sept_spot_15,10000000,1,NaN,2020-01-17,2020-01-17,True,True,4
74496,4317,Include,city,Kolkata,True,M_KOL,Men's Bengali_6,1000000,3,NaN,2020-01-10,2020-01-10,True,True,5


In [108]:
df2['impr_dur'] = df2.impr.astype(float) * df2.duration

/home/hadoop/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [123]:
df2['year'] = df2.min_cd.map(lambda x:x[:4])

/home/hadoop/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [111]:
df3 = df2.groupby('ad_set_id').agg({'ssai':lambda x: '/'.join(sorted(set(x))), 'impr_goal': sum, 'impr': sum, 'impr_dur': sum})
df3.to_csv('blaze-demand.csv')
df4 = df3.groupby('ssai').agg({'impr':sum, 'impr_goal': sum, 'impr_dur': sum}).reset_index().sort_values('impr_dur', ascending=False)
df4.to_csv('ssai_impr.csv', index=False)
df4

,ssai,impr,impr_goal,impr_dur
202,P_P_AN/P_P_ANT/P_P_IOS,506693969232,472372380060,9.844072e+12
268,P_P_AN/S_BR [Dep.]/S_BRJH/S_GJ [Dep.]/S_GJDD/S...,487689313900,586736424600,8.901854e+12
271,P_P_AN/S_BR [Dep.]/S_BRJH/S_GJ [Dep.]/S_GJDD/S...,267011725452,234027204586,4.402058e+12
203,P_P_AN/P_P_IOS,259190193276,271836304120,3.537992e+12
80,M_BEN/M_CHE/M_DEL/M_HYD/M_KOL/M_MUM/M_NCR/P_P_...,159598723653,165963804903,2.511774e+12
...,...,...,...,...
29,C2_1/C2_2,241568,1600000,7.005472e+06
132,M_BEN/M_MUM/M_NCR/M_PUN,367344,48000000,5.142816e+06
48,G_F/S_AP,226550,20000000,4.304450e+06
193,M_PUN,150569,40010000,3.183296e+06


In [114]:
df3

,ssai,impr_goal,impr,impr_dur
ad_set_id,,,,
637,M_BEN/M_CHE/M_HYD/M_KOL/M_MUM/M_NCR,3600000,6,84.0
689,S_WB,66667,11,99.0
1446,N_AT/N_NA,2,24,216.0
3369,C2_1/C2_NA,4,468,4212.0
3489,M_MUM/S_KL/S_TG,540000000,10144314,131876082.0
...,...,...,...,...
17035,P_P_AN/P_P_ANT/P_P_IOS,270000000,42028578,560381040.0
17037,P_P_AN/P_P_ANT/P_P_IOS,420000000,51425430,73464900.0
17040,P_P_AN/P_P_ANT/P_P_IOS,180000000,26501724,353356320.0


In [127]:
df3b = df2.groupby(['ad_set_id', 'year']).agg({'ssai':lambda x: '/'.join(sorted(set(x))), 'impr_goal': sum, 'impr': sum, 'impr_dur': sum})
df4b = df3b.groupby(['ssai', 'year']).agg({'impr':sum, 'impr_goal': sum, 'impr_dur': sum}).reset_index().sort_values('impr_dur', ascending=False)
df4b.to_csv('ssai_demand.csv', index=False)
df4b

,ssai,year,impr,impr_goal,impr_dur
345,P_P_AN/S_BR [Dep.]/S_BRJH/S_GJ [Dep.]/S_GJDD/S...,2021,487689313900,586736424600,8.901854e+12
269,P_P_AN/P_P_ANT/P_P_IOS,2022,262074709314,138264840060,5.601533e+12
270,P_P_AN/P_P_ANT/P_P_IOS,2023,244619259918,334107540000,4.242539e+12
348,P_P_AN/S_BR [Dep.]/S_BRJH/S_GJ [Dep.]/S_GJDD/S...,2021,254019751438,221351490645,4.160056e+12
271,P_P_AN/P_P_IOS,2022,255512336766,267897378812,3.491498e+12
...,...,...,...,...,...
254,M_PUN,2019,150569,40010000,3.183296e+06
501,S_KL,2021,4128,9019600,6.192000e+04
325,P_P_AN/S_APTG/S_BRJH/S_MH/S_OR/S_WB,2023,132,144000000,1.320000e+03
263,N_AT/N_NA,2019,24,2,2.160000e+02


In [137]:
df5 = df4b.pivot(['ssai'], columns='year', values=['impr_dur'])
df5

impr_dur                                           \
year                         2019         2020          2021          2022   
ssai                                                                         
A_U                           NaN          NaN  2.385520e+08           NaN   
A_U/P_P_AN                    NaN          NaN  7.287912e+09  1.476453e+09   
A_U/P_P_AN/P_P_IOS            NaN          NaN           NaN  1.850308e+10   
A_U30/P_P_AN/P_P_IOS          NaN          NaN           NaN  9.085535e+09   
C14_1                         NaN          NaN  2.927090e+10           NaN   
...                           ...          ...           ...           ...   
S_UP                  249196338.0          NaN           NaN           NaN   
S_UP [Dep.]/S_UPUT            NaN          NaN  1.906958e+08           NaN   
S_UPUT                        NaN          NaN  1.957412e+08           NaN   
S_WB                   42652047.0          NaN  3.090712e+08           NaN   
S_WBOR                        NaN  280752438.0           NaN           NaN   

                           
year                 2023  
ssai                       
A_U                   NaN  
A_U/P_P_AN            NaN  
A_U/P_P_AN/P_P_IOS    NaN  
A_U30/P_P_AN/P_P_IOS  NaN  
C14_1                 NaN  
...                   ...  
S_UP                  NaN  
S_UP [Dep.]/S_UPUT    NaN  
S_UPUT                NaN  
S_WB                  NaN  
S_WBOR                NaN  

[418 rows x 5 columns]

In [136]:
df6 = df4b.groupby('ssai').agg({'impr':sum, 'impr_goal': sum, 'impr_dur': sum})
df6

,impr,impr_goal,impr_dur
ssai,,,
A_U,11927600,11607142,2.385520e+08
A_U/P_P_AN,1045844978,1094353780,8.764365e+09
A_U/P_P_AN/P_P_IOS,873204663,884098062,1.850308e+10
A_U30/P_P_AN/P_P_IOS,302851179,302521005,9.085535e+09
C14_1,1540573649,4350000000,2.927090e+10
...,...,...,...
S_UP,27688482,110000,2.491963e+08
S_UP [Dep.]/S_UPUT,21188418,21052630,1.906958e+08
S_UPUT,18176965,20752394,1.957412e+08


In [143]:
df7=df6.join(df5).sort_values('impr_dur', ascending=False).fillna(0)
df7.to_csv('ssai_demand_year.csv')
df7

,impr,impr_goal,impr_dur,"(impr_dur, 2019)","(impr_dur, 2020)","(impr_dur, 2021)","(impr_dur, 2022)","(impr_dur, 2023)"
ssai,,,,,,,,
P_P_AN/P_P_ANT/P_P_IOS,506693969232,472372380060,9.844072e+12,0.0,0.0,0.000000e+00,5.601533e+12,4.242539e+12
P_P_AN/S_BR [Dep.]/S_BRJH/S_GJ [Dep.]/S_GJDD/S_GJDDGA/S_KA/S_KL/S_MH/S_MP [Dep.]/S_MPCT [Dep.]/S_MPCTRJ/S_PBHR [Dep.]/S_PBHRHPJK/S_PBHRHPJKCH/S_RJ [Dep.]/S_TN/S_UP [Dep.]/S_UPUT/S_WB,487689313900,586736424600,8.901854e+12,0.0,0.0,8.901854e+12,0.000000e+00,0.000000e+00
P_P_AN/S_BR [Dep.]/S_BRJH/S_GJ [Dep.]/S_GJDD/S_GJDDGA/S_KL/S_MH/S_MP [Dep.]/S_MPCT [Dep.]/S_MPCTRJ/S_PBHR [Dep.]/S_PBHRHPJK/S_PBHRHPJKCH/S_RJ [Dep.]/S_TN/S_UP [Dep.]/S_UPUT/S_WB,267011725452,234027204586,4.402058e+12,0.0,0.0,4.160056e+12,2.420013e+11,0.000000e+00
P_P_AN/P_P_IOS,259190193276,271836304120,3.537992e+12,0.0,0.0,0.000000e+00,3.491498e+12,4.649366e+10
M_BEN/M_CHE/M_DEL/M_HYD/M_KOL/M_MUM/M_NCR/P_P_AN/P_P_IOS,159598723653,165963804903,2.511774e+12,0.0,0.0,0.000000e+00,2.511774e+12,0.000000e+00
...,...,...,...,...,...,...,...,...
C2_1/C2_2,241568,1600000,7.005472e+06,7005472.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
M_BEN/M_MUM/M_NCR/M_PUN,367344,48000000,5.142816e+06,5142816.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
G_F/S_AP,226550,20000000,4.304450e+06,0.0,4304450.0,0.000000e+00,0.000000e+00,0.000000e+00
